In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, cross_val_score

from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [3]:
mass = pd.read_pickle('../data/03_dummy_df.pickle')

In [8]:
list(mass.columns)

['id',
 'country',
 'ccode',
 'region',
 'protestnumber',
 'protesterviolence',
 'location',
 'protesteridentity',
 'sources',
 'notes',
 'protester_id_type',
 'partipants_number',
 'protest_size_category',
 'pop_male',
 'pop_female',
 'pop_total',
 'pop_density',
 'prosperity_2020',
 'country_Afghanistan',
 'country_Albania',
 'country_Algeria',
 'country_Angola',
 'country_Argentina',
 'country_Armenia',
 'country_Austria',
 'country_Azerbaijan',
 'country_Bahrain',
 'country_Bangladesh',
 'country_Belarus',
 'country_Belgium',
 'country_Benin',
 'country_Bolivia',
 'country_Bosnia',
 'country_Botswana',
 'country_Brazil',
 'country_Bulgaria',
 'country_Burkina Faso',
 'country_Burundi',
 'country_Cambodia',
 'country_Cameroon',
 'country_Canada',
 'country_Cape Verde',
 'country_Central African Republic',
 'country_Chad',
 'country_Chile',
 'country_China',
 'country_Colombia',
 'country_Comoros',
 'country_Congo Brazzaville',
 'country_Congo Kinshasa',
 'country_Costa Rica',
 'coun

In [6]:
mass.shape

(15225, 207)

In [9]:
mass[[ 'protester_id_type']]

,protester_id_type
0,workers_unions
1,students_youth
2,political_group
3,ethnic_group
4,locals_residents
...,...
15220,locals_residents
15221,students_youth
15222,political_group
15223,political_group


In [13]:
mass[['protesterviolence']]

,protesterviolence
0,0.0
1,0.0
2,0.0
3,1.0
4,1.0
...,...
15220,1.0
15221,1.0
15222,0.0
15223,1.0


Dummify `protest_size_category` and `protester_id_type`  columns. 

In [15]:
mass = pd.get_dummies(mass, columns=['protest_size_category', 'protester_id_type'], dummy_na=True)

In [35]:
# list(mass.columns)

Dropping `nan` columns created from dummifying `protest_size_category` and `protester_id_type`  columns. 

In [19]:
mass.drop(columns=['protest_size_category_nan', 'protester_id_type_nan'], inplace=True)

In [55]:
features = mass.drop(columns=['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
                              'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings'])

In [56]:
# mass['start_date']

Below, attempting a Logistic Regression for each state response, starting with `arrests`.

In [64]:
X = mass.drop(columns=['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
                              'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings'])
y = mass['arrests']

In [65]:
type(y)

pandas.core.series.Series

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [67]:
ss = StandardScaler()

X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [70]:
lr = LogisticRegression(solver='lbfgs', max_iter=100)

lr.fit(X_train_sc, y_train)

/Users/emilynaftalin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [72]:
lr.score(X_train_sc, y_train)

0.8637931034482759

In [73]:
lr.score(X_test_sc, y_test)

0.851888341543514

In [74]:
lr.predict(X_test_sc)

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
class StateResponsePredictor:
    
    def __init__(self):
        self.state_response_list = state_response_list
        
    def fit(self, mass):
        self.ss = StandardScaler()
        X = mass.drop(columns=['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
                              'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']) 
        

In [89]:
# drop_from_X = ['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
#                               'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']
# state_responses = ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']

# response_dict = {}

def state_response_predictor(df):
    
    drop_from_X = ['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
                              'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']
    state_responses = ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']

    response_dict = {}
    
    X = df.drop(columns = drop_from_X)
    
    ss = StandardScaler()

    
    
    for response in state_responses: 
        
        y = mass[response]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)
        
        X_train_sc = ss.fit_transform(X_train)
        X_test_sc = ss.transform(X_test)
        
        lr = LogisticRegression(solver='lbfgs', max_iter=100)

        lr.fit(X_train_sc, y_train)
        
        response_dict[response] = lr.predict_proba(X_test_sc)[0][1]
        print(response_dict)
        
    top_response = max(response_dict.items(), key=lambda k: k[1])
    
    return top_response 

In [90]:
state_response_predictor(mass)

/Users/emilynaftalin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'arrests': 0.4016257727256075}
{'arrests': 0.4016257727256075, 'accomodation': 0.07206943121944955}


/Users/emilynaftalin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'arrests': 0.4016257727256075, 'accomodation': 0.07206943121944955, 'beatings': 0.21386391025228904}


/Users/emilynaftalin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'arrests': 0.4016257727256075, 'accomodation': 0.07206943121944955, 'beatings': 0.21386391025228904, 'crowddispersal': 0.3818345169499125}
{'arrests': 0.4016257727256075, 'accomodation': 0.07206943121944955, 'beatings': 0.21386391025228904, 'crowddispersal': 0.3818345169499125, 'ignore': 0.07664528924354543}


/Users/emilynaftalin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'arrests': 0.4016257727256075, 'accomodation': 0.07206943121944955, 'beatings': 0.21386391025228904, 'crowddispersal': 0.3818345169499125, 'ignore': 0.07664528924354543, 'killings': 0.35658368862007894}
{'arrests': 0.4016257727256075, 'accomodation': 0.07206943121944955, 'beatings': 0.21386391025228904, 'crowddispersal': 0.3818345169499125, 'ignore': 0.07664528924354543, 'killings': 0.35658368862007894, 'shootings': 0.3092331274246054}


/Users/emilynaftalin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


('arrests', 0.4016257727256075)

In [ ]:
mass['is_arrests'] = []